In [9]:
!pip install sentence-transformers

In [8]:
import pandas as pd
import re
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer
import ast
import faiss
# import json

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

# 1) Load to Memory & Merge Features and Desc.

In [5]:
games_df = pd.read_csv('games.csv')
metadata_df = pd.read_json('games_metadata.json', lines=True)

In [ ]:
df = pd.merge(games_df, metadata_df, on='app_id', how='inner')

In [11]:
games_df = None
metadata_df = None

In [ ]:
# df.to_csv('Altered CSVs/merged_game_data.csv', index=False)

# 2.1) One Hot Encoding (Deprecated)

In [ ]:
# df = pd.read_csv('Altered CSVs/merged_game_data.csv')

In [3]:
df['tags']

0        ['Action', 'Adventure', 'Parkour', 'Third Pers...
1                                               ['Action']
2        ['Co-op', 'Stealth', 'Indie', 'Heist', 'Local ...
3        ['Zombies', 'Adventure', 'Survival', 'Action',...
4        ['Roguelike', 'Strategy', 'Tower Defense', 'Pi...
                               ...                        
50867                                                   []
50868                                                   []
50869                                                   []
50870    ['Early Access', 'FPS', 'Action', 'Retro', 'Fi...
50871                                                   []
Name: tags, Length: 50872, dtype: object

In [4]:
# type(df['tags'][0])
    # Tags are stored in a str format*

In [3]:
# Iterate through tags and produce a list of all tags -- with repeats
tags_array = []
for tags in df['tags']:
    for tag in tags.replace('[','').replace(']','').replace('\'','').replace('\"','').split(','):
        tags_array.append(tag)

In [4]:
# Keep only unique tags and Remove Empty Tags
unique_tags = set(tags_array)
unique_tags.remove('')

In [ ]:
# Remove Random Whitespace at the Start of Tags & Produce One-Hot Encoding Columns
for tag in unique_tags:
    if(tag[0]==' '):
        tag = tag[1:]   
    df[tag] = 0

In [ ]:
df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,0,0,0,0,0,0
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,0,0,0,0,0,0,0,0
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,...,0,0,0,0,0,0,0,0,0,0
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,...,0,0,0,0,0,0,0,0,0,0
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,...,0,0,0,0,0,0,0,0,0,0
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,...,0,0,0,0,0,0,0,0,0,0


In [10]:
for index, row in df.iterrows():
    for tag in row['tags'].replace('[','').replace(']','').replace('\'','').replace('\"','').split(','):
        tag = tag.strip()
        if tag in unique_tags:
            df.at[index, tag] = 1
df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,0,0,0,0,0,0
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,0,0,0,0,0,0,0,0
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,...,0,0,0,0,0,0,0,0,0,0
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,...,0,0,0,0,0,0,0,0,0,0
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,...,0,0,0,0,0,0,0,0,0,0
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df.to_csv('Altered CSVs/one_hot_encoded_tags.csv', index=False)

# 2.2) MultiLabelBinarizer

In [ ]:
# df = pd.read_csv('Altered CSVs/merged_game_data.csv')

In [41]:
df

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,0,0,0,0,0,0
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,0,0,0,0,0,0
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,0,0,0,0,0,0
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,0,0,0,0,0,0,1,0
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,...,0,0,0,0,0,0,0,0,0,0
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,...,0,0,0,0,0,0,0,0,0,0
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,...,0,0,0,0,0,0,0,0,0,0
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,...,0,0,0,0,0,0,0,0,0,0


In [16]:
mlb = MultiLabelBinarizer()

In [34]:
# Convert the 'tags' column from a string representation of a list to an actual list
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [35]:
tag_matrix = mlb.fit_transform(df['tags'])

In [39]:
tags_df = pd.DataFrame(tag_matrix, columns=mlb.classes_)

In [40]:
df = pd.concat([df, tags_df], axis=1)

In [ ]:
# df.to_csv('Altered CSVs/MutliLabelBinarized.csv', index=False)

# 3.1) TFIDF (Deprecated)

## Preprocessing & Cleaning

In [ ]:
# df = pd.read_csv('Altered CSVs/one_hot_encoded_tags.csv')

### Attempt 1 Remenants

*Remenants of a previous attempt at preprocessing & cleaning; kept for reference in second attempt*

In [ ]:
# # Print a list with the names of empty descriptions columns
# empty_descriptions = df[df['description'].isnull()]
# empty_descriptions_list_name = empty_descriptions['title'].tolist()
# empty_descriptions_list_id = empty_descriptions['app_id'].tolist()
# empty_descriptions_list_name[0:5], empty_descriptions_list_id[0:5]

(['BRINK: Agents of Change',
  'Men of War: Assault Squad 2 - Deluxe Edition upgrade',
  'Borderlands 2: Headhunter 4: Wedding Day Massacre',
  'Sniper Elite 3 - Camouflage Weapons Pack',
  'The House in Fata Morgana Original Soundtrack'],
 [22364, 271850, 245950, 305181, 458790])

In [ ]:
# # Some Probelem Descriptions that have just been difficult working with*
# arr =  [1456880, 342570, 930840, 1837980]
# df[df['app_id'].isin(arr)][['app_id','title','description']]

,app_id,title,description
14865,1456880,ElecHead,⚡＋🤖＋🧩
25173,342570,HIS (Heroes In the Sky),-
29524,930840,东方百问~TouHouAsked,Null
44906,1837980,Our Elusive Suffering,🩸🌌💀🌫️⛪️


In [ ]:
# # Replace missing descriptions with the title and tags
# df['description'] = df['description'].fillna(df['title'] + df['tags'])

In [5]:
# # Remove 'tags' column
# df = df.drop(columns=['tags'])

In [ ]:
# # Verify that the descriptions are no longer empty
# df[df['description'].isnull()]

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up


In [ ]:
# # Show App IDs from array
# arr = [22364, 271850, 245950, 305181, 458790]
# df[df['app_id'].isin(arr)][['app_id','title','description']]

,app_id,title,description
1,22364,BRINK: Agents of Change,BRINK: Agents of Change ['Action']
7,271850,Men of War: Assault Squad 2 - Deluxe Edition u...,Men of War: Assault Squad 2 - Deluxe Edition u...
18,245950,Borderlands 2: Headhunter 4: Wedding Day Massacre,Borderlands 2: Headhunter 4: Wedding Day Massa...
21,305181,Sniper Elite 3 - Camouflage Weapons Pack,Sniper Elite 3 - Camouflage Weapons Pack ['Adv...
30,458790,The House in Fata Morgana Original Soundtrack,The House in Fata Morgana Original Soundtrack ...


In [ ]:
# df['description']

0        Enter the dark underworld of Prince of Persia ...
1                       BRINK: Agents of Change ['Action']
2        Monaco: What's Yours Is Mine is a single playe...
3        Escape Dead Island is a Survival-Mystery adven...
4        Dungeon of the Endless is a Rogue-Like Dungeon...
                               ...                        
50867         I Expect You To Die 3: Cog in the Machine []
50868                                          PAYDAY 3 []
50869                                        Eternights []
50870    Embark on a journey into the darkest nightmare...
50871                                         FatalZone []
Name: description, Length: 50872, dtype: object

In [ ]:
# df.to_csv('Altered CSVs/before_tokenization.csv', index=False)

In [ ]:
# df = pd.read_csv('Altered CSVs/before_tokenization.csv')

In [ ]:
# # Check for na values in the description column
# df['description'].isna().sum()

4

In [ ]:
# def clean_text(text):
#     text = re.sub(r'[^\w\s]', '', text)
#     text = text.lower()
#     text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
#     return text

In [ ]:
# # Remove Special Character and Punctuation from Descriptions
# df['description'] = df['description'].apply(clean_text)

In [ ]:
# empty_descriptions = df[df['description'].isnull()]
# empty_descriptions_list_name = empty_descriptions['title'].tolist()
# empty_descriptions_list_id = empty_descriptions['app_id'].tolist()
# empty_descriptions_list_name, empty_descriptions_list_id

([], [])

In [ ]:
# df['tokens'] = df['description'].apply(word_tokenize)

In [ ]:
# df['tokens'].head()

0    [Enter, the, dark, underworld, of, Prince, of,...
1     [BRINK, :, Agents, of, Change, [, 'Action, ', ]]
2    [Monaco, :, What, 's, Yours, Is, Mine, is, a, ...
3    [Escape, Dead, Island, is, a, Survival-Mystery...
4    [Dungeon, of, the, Endless, is, a, Rogue-Like,...
Name: tokens, dtype: object

In [ ]:
# arr =  [1456880, 342570, 930840, 1837980]
# df[df['app_id'].isin(arr)][['app_id','title','description','tokens']]

,app_id,title,description,tokens
14865,1456880,ElecHead,,[]
25173,342570,HIS (Heroes In the Sky),,[]
29524,930840,东方百问~TouHouAsked,null,[null]
44906,1837980,Our Elusive Suffering,,[]


### Fixed Cleaning

In [6]:
# Fill Empty Descriptions
df['description'] = df['description'].fillna(df['title'] +' '+ df['tags'])

In [7]:
def clean_text(text):
    # Remove Special Characters
    text = re.sub(r'[^\w\s]', '', text)

    # Sets all characters to lowercase
    text = text.lower()

    # Removes URLs beginning with https, http, or www
    text = re.sub(r'https\S+|http\S+|www\S+', '', text, flags=re.MULTILINE)
    
    return text

In [8]:
df['description'] = df['description'].apply(clean_text)

In [9]:
# Print any rows with descriptions with only whitespace characters
empty_descriptions = df[df['description'].str.strip() == '']
empty_descriptions

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
14865,1456880,ElecHead,2021-10-14,True,False,False,Overwhelmingly Positive,98,588,9.99,...,0,0,0,0,0,0,0,0,0,0
25173,342570,HIS (Heroes In the Sky),2015-03-03,True,False,False,Mostly Negative,38,539,0.00,...,0,0,0,1,0,0,0,0,0,0
44906,1837980,Our Elusive Suffering,2022-03-26,True,False,False,Mixed,68,70,0.00,...,0,0,0,0,0,0,0,0,0,0


In [10]:
null_literal_descriptions = df[df['description'].str.strip() == 'null']
null_literal_descriptions

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up
29524,930840,东方百问~TouHouAsked,2018-10-07,True,False,False,Positive,93,29,0.99,...,0,1,0,0,0,0,0,0,0,0


Refill Empty, Whitespace Descriptions ~~and Null Literal Descriptions~~

In [11]:
rows = df['description'].str.strip() == ''
df.loc[rows, 'description'] = df.loc[rows, 'title'] +' '+ df.loc[rows, 'tags']

In [12]:
# Apply Cleaning to Whitespace Columns
df.loc[rows, 'description'] = df.loc[rows, 'description'].apply(clean_text)

In [13]:
# Reprint any rows with descriptions with only whitespace characters
empty_descriptions = df[df['description'].str.strip() == '']
empty_descriptions

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,Competitive,Faith,8-bit Music,Arcade,Relaxing,Lemmings,Emotional,1990s,Beat em up,Shoot Em Up


In [14]:
# Print Fixed Descriptions
df.loc[rows, 'description']

14865    elechead puzzle platformer puzzle platformer l...
25173    his heroes in the sky free to play multiplayer...
44906    our elusive suffering horror surreal explorati...
Name: description, dtype: object

In [ ]:
# Checking Random Description Columns
pd.set_option('display.max_colwidth', None)
print(df['description'].sample(n=3))
pd.reset_option('display.max_colwidth')

In [ ]:
# df.to_csv('Altered CSVs/after_cleaning.csv', index=False)

## Tokenization

In [ ]:
# df = pd.read_csv('Altered CSVs/after_cleaning.csv')

In [18]:
# Tokenize the descriptions
df['tokens'] = df['description'].apply(word_tokenize)

In [21]:
# Remove Stop Words from Tokens
stop_words = set(stopwords.words('english'))

df['tokens'] = df['tokens'].apply(
    lambda tokens: [word for word in tokens if word not in stop_words]
)

In [23]:
# Lemmatize the Tokens
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(
    lambda tokens: [lemmatizer.lemmatize(word) for word in tokens]
)

In [ ]:
# df.to_csv('Altered CSVs/after_tokenization.csv', index=False)

# 3.2) Sentance Transformer / Embeddings



In [ ]:
# df = pd.read_csv('Altered CSVs/MutliLabelBinarized.csv')

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [11]:
df['title'] = df['title'].fillna('')
df['description'] = df['description'].fillna('')

In [12]:
df['text_for_embedding'] = df['title'] + ' ' + df['description'] + ' ' + df['tags']

In [16]:
df['embedding'] = list(text_embeddings)

In [ ]:
# df.to_csv('Altered CSVs/with_embeddings.csv', index=False)

# 4.1) Cosine Similarity Search

In [ ]:
# df = pd.read_csv('Altered CSVs/after_tokenization.csv')

In [29]:
# Glue Tokens
df['tokens'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

In [30]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tokens'])

In [35]:
def get_similar_games(app_id, top_n):
    # Get Description for the given app_id
    query = df[df['app_id'] == app_id]['tokens'].values[0]
    query_vector = tfidf_vectorizer.transform([query])

    # Compute cosine similarity
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get top_n indices of similar games; grabs the last n indexes from the sorted array and then flips them
    top_indices = cosine_similarities.argsort()[-top_n-1:-1][::-1]

    # Retrieve game titles and app_ids for the top similar games
    similar_games = df.iloc[top_indices][['app_id', 'title', 'tokens']]
    similar_games['similarity'] = cosine_similarities[top_indices]
    similar_games = similar_games.sort_values(by='similarity', ascending=False)

    # Return the DataFrame with similar games
    return similar_games

In [36]:
# Testing with SH:MCD
similar_games = get_similar_games(690040, top_n=6)
similar_games

,app_id,title,tokens,similarity
28262,1261380,SUPERHOT: MIND CONTROL DELETE Soundtrack,superhot mind control delete soundtrack action...,0.461258
42813,652910,Delay,inspired superhot delay fastpaced action platf...,0.217625
40651,513620,SkyTime,skytime 3d platformer slow time dodge bullet d...,0.207488
47656,322500,SUPERHOT,superhot smashhit fps time move move regenerat...,0.183004
47432,24790,Command & Conquer 3: Tiberium Wars,year 2047 massive nuclear fireball explodes hi...,0.178755
47958,617830,SUPERHOT VR,lose track whats real commit body mind confron...,0.168865


In [37]:
# Testing with Elden Ring
similar_games = get_similar_games(1245620, top_n=6)
similar_games

,app_id,title,tokens,similarity
32042,1070320,Ring the City,wear ring fight many ring wear one hand ring c...,0.502929
47516,214510,LEGO® The Lord of the Rings™,based lord ring motion picture trilogy lego lo...,0.411813
13168,212500,The Lord of the Rings Online™,lord ring online,0.396808
10945,998740,Ring of Pain,enter ring pain roguelike card crawler encount...,0.303529
2390,2218370,Thunderbolt Poker,collect ring fragment forge legend ring treasu...,0.301577
36922,715020,Elden: Path of the Forgotten,guide elden mission across blighted land save ...,0.298580


# 4.2) Normalization

## Min-Max

In [ ]:
# df = pd.read_csv('Altered CSVs/with_embeddings.csv')

**Applied for Ratings**

In [3]:
rating_dict = {
    'Overwhelmingly Positive': 9,
    'Very Positive': 8,
    'Positive': 7,
    'Mostly Positive': 6,
    'Mixed': 5,
    'Mostly Negative': 4,
    'Negative': 3,
    'Very Negative': 2,
    'Overwhelmingly Negative': 1
}


In [5]:
df['rating_normalized'] = (df['rating'].map(rating_dict) - 1) / 8

## Log Normalizing

**Applied for Positive Ratio, Price, and Review Count**

In [11]:
df['positive_ratio_log'] = np.log1p(df['positive_ratio'])
df['price_log'] = np.log1p(df['price_final'])
df['user_reviews_log'] = np.log1p(df['user_reviews'])

In [ ]:
# df.to_csv('Altered CSVs/normalized.csv', index=False)

# 5.2) Concatenating Features

In [22]:
df.head(5)

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,...,World War II.1,Wrestling.1,Zombies.1,eSports.1,text_for_embedding,embedding,rating_normalized,positive_ratio_log,price_log,user_reviews_log
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,...,0,0,0,0,Prince of Persia: Warrior Within™ Enter the da...,[-4.18693461e-02 -5.43150157e-02 2.60398388e-...,0.875,4.442651,2.396986,7.696213
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,...,0,0,0,0,BRINK: Agents of Change ['Action'],[-2.81047057e-02 1.77983008e-02 -6.03910424e-...,0.750,4.454347,1.383791,3.091042
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,...,0,0,0,0,Monaco: What's Yours Is Mine Monaco: What's Yo...,[ 9.24706180e-03 -7.27140531e-02 -3.38471904e-...,0.875,4.532599,2.771964,8.222285
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,...,0,0,1,0,Escape Dead Island Escape Dead Island is a Sur...,[-1.55046768e-02 -8.70501176e-02 -7.24602565e-...,0.500,4.127134,2.771964,6.773080
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,...,0,0,0,0,Dungeon of the ENDLESS™ Dungeon of the Endless...,[ 7.03207683e-04 -5.76260015e-02 1.02267154e-...,0.875,4.488636,2.564180,9.080801


In [20]:
df = pd.read_csv('Altered CSVs/normalized.csv')

In [29]:
numerics = df[['price_log', 'positive_ratio_log', 'user_reviews_log']]

In [ ]:
X = np.hstack([numerics])